In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import time
import cv2

from joblib import Parallel, delayed

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from data_manager import *
from metadata import *

import matplotlib.pyplot as plt
%matplotlib inline

from skimage.transform import rotate

In [ ]:
cell_masks_normalized = bp.unpack_ndarray_file('/home/yuncong/csd395/CSHL_cells/fractal_dim/cell_masks_normalized.bp')

In [ ]:
n_cells = len(cell_masks_normalized)
print n_cells, 'cells'

cell_masks_normalized_flattened = np.reshape(cell_masks_normalized, (len(cell_masks_normalized), -1))
cell_masks_normalized_flattened.shape

cell_masks_normalized_size = cell_masks_normalized_flattened.sum(axis=1)

In [ ]:
memberCount = np.bincount(indices_of_closest_seed, minlength=len(seeds))
seedIndices_sorted_by_memberCount = np.argsort(memberCount)[::-1]
memberCount_sorted = memberCount[seedIndices_sorted_by_memberCount]
seeds_ranked_by_memberCount = seeds[seedIndices_sorted_by_memberCount]

In [ ]:
plt.plot(np.cumsum(memberCount_sorted) / float(n_cells));
plt.xlabel('top # seeds');
plt.ylabel('percentage coverage');

In [ ]:
from multiprocess import Pool
from scipy.spatial.distance import squareform

def compute_jaccard_list_with_all(seed_indices):

    pool = Pool(14)
    affinities_to_seeds = pool.map(compute_jaccard_with_i, seed_indices)
    pool.close()
    pool.join()
    return np.asarray(affinities_to_seeds)

def compute_jaccard_with_i_list(i, indices):
    intersections = cell_masks_normalized_flattened[indices[:,None], cell_masks_normalized_flattened[i]].sum(axis=1)
    unions = cell_masks_normalized_size[i] + cell_masks_normalized_size[indices] - intersections
    scores = intersections.astype(np.float)/unions
    return scores
    
def compute_jaccard_pairwise(indices, square_form=True, parallel=True):
    n = len(indices)

    if parallel:
        pool = Pool(16)
        pairwise_scores = pool.map(lambda x: compute_jaccard_with_i_list(x[0],x[1]), 
                                   [(indices[i], indices[i+1:]) for i in range(n)])
        pool.close()
        pool.join()
    else:
        pairwise_scores = [compute_jaccard_with_i_list(indices[i], indices[i+1:]) for i in range(n)]
        
    if square_form:
        return squareform(np.concatenate(pairwise_scores))
    else:
        return pairwise_scores

def compute_jaccard_with_i(i, upper=False):
    if upper:
        intersections_with_i = cell_masks_normalized_flattened[i+1:, cell_masks_normalized_flattened[i]].sum(axis=1)
        unions_with_i = cell_masks_normalized_size[i] + cell_masks_normalized_size[i+1:] - intersections_with_i
    else:
        intersections_with_i = cell_masks_normalized_flattened[:, cell_masks_normalized_flattened[i]].sum(axis=1)
        unions_with_i = cell_masks_normalized_size[i] + cell_masks_normalized_size - intersections_with_i
        
    return intersections_with_i.astype(np.float)/unions_with_i

def compute_jaccard_with_template(template):
    intersections_with_template = [template[m].sum() for m in cell_masks_normalized_flattened]
    unions_with_template = (template + cell_masks_normalized_size - intersections_with_template)
    return intersections_with_template.astype(np.float)/unions_with_template

def compute_jaccard_with_i_sparse(i, upper=False, threshold=.85, n_neighbors=10):
    if upper:
        scores = compute_jaccard_with_i(i, upper=True)
        nearest_neighbors = np.where(scores > threshold)[0]
        return i+1+nearest_neighbors, scores[nearest_neighbors]
    else:
        scores = compute_jaccard_with_i(i, upper=False)
        nearest_neighbors = np.argsort(scores)[::-1][:10]
        return nearest_neighbors, scores[nearest_neighbors]

In [ ]:
# Method 1

In [ ]:
data_per_batch = 1000
n_batches = n_cells / data_per_batch
print 'n_batches =', n_batches
slave_start_indices = np.linspace(0, n_cells, n_batches+1).astype(np.int)

for batch_index in range(n_batches):
    
    print batch_index
    t = time.time()

    begin_data_index = slave_start_indices[batch_index]
    end_data_index = slave_start_indices[batch_index+1]-1
    scores = compute_jaccard_list_with_all(range(begin_data_index, end_data_index+1))
#     scores[scores < threshold] = 0
    bp.pack_ndarray_file(scores.astype(np.float16), '/home/yuncong/csd395/CSHL_cells/fractal_dim/pairwise_scores_%d_%d.bp' % (begin_data_index,
                                                                                                          end_data_index))
    
    sys.stderr.write('Compute pairwise affinities: %f s.\n' % (time.time()-t)) # 44s / 1000 x 200k

In [ ]:
# Sparse

affinities_mat_full = dok_matrix((n_cells, n_cells), dtype=np.float16)

data_per_batch = 1000
n_batches = n_cells / data_per_batch
print 'n_batches =', n_batches
slave_start_indices = np.linspace(0, n_cells, n_batches+1).astype(np.int)

threshold = 0.8

for batch_index in range(n_batches):
    
    print batch_index
    t = time.time()

    begin_data_index = slave_start_indices[batch_index]
    end_data_index = slave_start_indices[batch_index+1]-1
    
    scores = bp.unpack_ndarray_file('/home/yuncong/csd395/CSHL_cells/fractal_dim/pairwise_scores_%d_%d.bp' % (begin_data_index,
                                                                                                          end_data_index))
    
    sys.stderr.write('Load pairwise affinities: %f s.\n' % (time.time()-t)) # 15s / 1000 x 200k
    
    t = time.time()
    
#     nearest_neighbors_cols = np.argsort(scores, axis=1)[:, ::-1][:, :10]
#     for i in range
    
    nearest_neighbors_rows, nearest_neighbors_cols = np.where(scores > threshold)
    
    sys.stderr.write('Load pairwise affinities: %f s.\n' % (time.time()-t)) # 15s / 1000 x 200k
    
    t = time.time()
    affinities_mat_full[begin_data_index + nearest_neighbors_rows, nearest_neighbors_cols] = scores[nearest_neighbors_rows, nearest_neighbors_cols]
    
    sys.stderr.write('Load pairwise affinities: %f s.\n' % (time.time()-t)) # 15s / 1000 x 200k
    
    break

In [ ]:
import scipy.sparse.linalg

D = np.sum(affinities_mat_full, axis=1)
L = D - affinities_mat_full
eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(L, M=D)
embedding = eigenvectors[:7, :]

In [ ]:
# Full Spectral Clutering

import scipy.linalg

distance_mat_full = squareform(pdist(data))
affinities_mat_full = np.exp(-distance_mat_full**2/10.)

D = np.diag(np.sum(affinities_mat_full, axis=1))
L = D - affinities_mat_full
eigenvalues, eigenvectors = scipy.linalg.eigh(L, D)
nvec = 2
E_original_order = eigenvectors[:, 1:1+nvec]

In [ ]:
# scatter plot

from sklearn.cluster import KMeans, MiniBatchKMeans

kmeans = KMeans(n_clusters=n_classes)
kmeans.fit(E_original_order);

print np.bincount(kmeans.labels_, minlength=n_classes)

for i in range(n_classes):
    indices = np.where(kmeans.labels_ == i)[0]
    plt.scatter(data[indices,0], data[indices,1], c=colors[i]);
plt.show();

In [ ]:
# our data, Nystroem extension
# https://people.eecs.berkeley.edu/~malik/papers/FBCM-nystrom.pdf

t = time.time()

n_seeds = 200
sampled = seeds_ranked_by_memberCount[:n_seeds].copy()

affinities_with_samples = compute_jaccard_list_with_all(sampled)

sys.stderr.write('Compute pairwise affinities (with samples): %f s.\n' % (time.time()-t))

In [ ]:
nonsampled = np.setdiff1d(range(affinities_with_samples.shape[1]), sampled)
permutation = np.r_[sampled, nonsampled]

In [ ]:
t = time.time()

from scipy.linalg import sqrtm

A = affinities_with_samples[:, sampled].copy()
B = affinities_with_samples[:, nonsampled].copy()

d1 = np.sum(np.c_[A, B], axis=1)
Ai = np.linalg.inv(A)
d2 = np.sum(B, axis=0) + np.dot(B.T, np.dot(Ai, np.sum(B, axis=1)))
dhat_si = np.sqrt(1./np.r_[d1, d2])
A = A*np.outer(dhat_si[:n_seeds], dhat_si[:n_seeds])
B = B*np.outer(dhat_si[:n_seeds], dhat_si[n_seeds:])

Asi = sqrtm(Ai)

M = np.dot(B.T, Asi)
S = A + np.dot(M.T, M)

U, L, T = np.linalg.svd(S)

V = np.dot(np.vstack([A, B.T]), np.dot(Asi, np.dot(U, np.linalg.inv(np.diag(np.sqrt(L))))))

sys.stderr.write('Nystroem: %f s.\n' % (time.time()-t)) # 60s / 100 samples

In [ ]:
E_allEig = V[:,1:]/V[:,0][:,None]

In [ ]:
E_allEig_original_order = np.zeros_like(E_allEig)
E_allEig_original_order[permutation] = E_allEig

bp.pack_ndarray_file(E_allEig_original_order, 
                     '/home/yuncong/csd395/CSHL_cells/fractal_dim/embeddingAllEigen_nystromSample%d.bp' % n_seeds)

# print E_allEig_original_order.mean(axis=0)
# print E_allEig_original_order.std(axis=0)

E_allEig_original_order_normalized = (E_allEig_original_order-E_allEig_original_order.mean(axis=0))/E_allEig_original_order.std(axis=0)

bp.pack_ndarray_file(E_allEig_original_order_normalized, 
                     '/home/yuncong/csd395/CSHL_cells/fractal_dim/embeddingAllEigenNormalized_nystromSample%d.bp' % n_seeds)